In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection

/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection


In [3]:
%cd models/research
# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/models/research
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 24.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
    

In [4]:
!python object_detection/builders/model_builder_tf2_test.py

2023-05-01 20:08:21.780810: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 20:08:23.837921: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2023-05-01 20:08:29.950932: I tensorflow/compil

In [2]:
#importing necessary libraries
import glob
import cv2
import time
import numpy as np
import os
import imageio
from imutils.video import FPS
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util


In [3]:
#load the pretrained model
start_time = time.time()
tf.keras.backend.clear_session()
detect_fn = tf.saved_model.load('/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/exportmodel_waymo_v120_efficientdet_d4/saved_model')
end_time = time.time()
elapsed_time = end_time - start_time
print('Elapsed time: ' + str(elapsed_time) + 's')

Elapsed time: 56.69955897331238s


In [4]:
#load the label map for the dataset
label_map_path='/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/waymo_labelmap.txt'
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [5]:
#navigate to the directory which contains the tracker.py file
%cd /content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/diploma/tracking

/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/diploma/tracking


In [9]:
#to resolve the numpy compiled against different versions error when you import Tracker
!pip install --upgrade numpy
!pip install --upgrade tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 37.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      

In [23]:
from utils.tracker import Tracker
import cv2
import datetime
import argparse
import six

In [28]:
#testing on videos
directory = '/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/data/data/FromYoutube/'

# initialize tracker
non_active_threshold = 15
active_threshold = 8
tracker = Tracker(active_threshold=active_threshold, non_active_threshold=non_active_threshold)
prev_dist=0
prev_id = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        output_video_path = f
        #open the video
        cap = cv2.VideoCapture(output_video_path)
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        fps = FPS().start()

        #set up the object for writing the created output video
        res_path = '/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/results/lane/'
        out = cv2.VideoWriter(res_path+'1'+filename, cv2.VideoWriter_fourcc('M','P','4','V'), 30, (frame_width,frame_height))

        currentframe = 0
        while (cap.isOpened() and currentframe<2000):
            ret, image_np = cap.read()
            if len((np.array(image_np)).shape) == 0:
                break
            input_tensor = np.expand_dims(image_np, 0)

            #get the detections
            detections = detect_fn(input_tensor)        
            label_id_offset = 1

            #create a copy of the input image to write to the output
            image_np_with_detections = image_np.copy()

            #writing the detections onto the image
            #first we need to get the boxes that are actually used
            boxes = detections['detection_boxes'][0].numpy()

            # get all boxes from an array
            max_boxes_to_draw = boxes.shape[0]
            # get scores to get a threshold
            scores = detections['detection_scores'][0].numpy()
            min_score_thresh=.5

            #creating lists to store the detections that have a confidence value that is higher than the min_score_thresh
            output_boxes=[]
            output_scores=[]
            output_classes=[]
            # iterate over all objects found
            for i in range(min(max_boxes_to_draw, boxes.shape[0])):                
                if scores[i] > min_score_thresh:
                    # boxes[i] is the box which will be drawn        
                    ymin, xmin, ymax, xmax = boxes[i]

                    #converting from xywh to xyxy
                    left = xmin * frame_width
                    top = ymin * frame_height
                    right = xmax * frame_width
                    bottom = ymax * frame_height
                    output_boxes.append((int(left),int(top),int(right),int(bottom)))
                    output_classes.append(detections['detection_classes'][0][i].numpy().astype(np.int32))
                    output_scores.append(scores[i])

            #update the tracker        
            objects, tracked_classes = tracker.update(output_boxes, output_classes)            
            ids = {}

            #visualisation
            for i,(object_id, centroid) in enumerate(objects.items()):
              text = "id{}".format(object_id)
              class_name = str(category_index[tracked_classes[object_id]]['name'])
              f = 250 #focal length of the camera

              #assumptions are made about the original height of the object so that the distance between the camera and the object can be calculated
              if class_name == 'vehicle':
                color = (255,255,0)
                original_height = 60
              elif class_name == 'pedestrian':
                color = (0,255,255)
                original_height = 63
              else:
                color = (255,0,255)
                original_height = 60

              #distance estimation
              dist = (original_height * f)/ (centroid[3] - centroid[1])
              dist = round(dist/(12*3.281),5)

              #to mark the vehicles coming too close, set the bbox colour to red
              if dist<=5:
                color = (0,0,255)
              
              if object_id in prev_id.keys():
                #define the region of interest (center of the frame) and if the vehicle lies between that then calculate its velocity and visualise its bbox in a different colour
                midx = (centroid[0] + centroid[2])//2
                if midx>150 and midx<350:
                  color = (0,255,0)
                  #check if this object has been tracked in the prev frame, if it has then retrieve that distance and estimate its velocity
                  if prev_id[object_id] !=0 and currentframe%25==0:
                    velo = abs((prev_id[object_id] - dist)) * 25 * 3.6 #assuming FPS is 25                  
                    
                    if velo > 80:
                      init_text = 'Warning'
                    else:
                      init_text = 'The'
                    if velo!=0:
                      print(f'{init_text} Velocity = {velo:.3f} km/hr')
                    prev_dist = dist
              ids[object_id]=dist 

              cv2.rectangle(image_np_with_detections, (centroid[0], centroid[1]), (centroid[2], centroid[3]), color, 2)
              cv2.rectangle(image_np_with_detections, (centroid[0], centroid[1]-30), (centroid[0]+(len(class_name))*3, centroid[1]), color, -1)
              cv2.putText(image_np_with_detections, class_name[0]+"-"+str(round(dist,2))+"m ",(centroid[0], centroid[1]-10),0, 0.75, (255,255,255),2)
              cv2.putText(image_np_with_detections, text,(centroid[2], centroid[3]),0, 0.75, (255,255,255),2)         
              cv2.circle(image_np_with_detections, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

            prev_id = ids

            currentframe += 1
            fps.update()
            out.write(image_np_with_detections)
            
        fps.stop()
        print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
        print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
        cap.release()
        out.release()

The Velocity = 15.746 km/hr
The Velocity = 4.229 km/hr
The Velocity = 1.449 km/hr
The Velocity = 4.253 km/hr
The Velocity = 0.906 km/hr
The Velocity = 10.195 km/hr
The Velocity = 27.212 km/hr
The Velocity = 32.423 km/hr
The Velocity = 32.470 km/hr
The Velocity = 26.375 km/hr
The Velocity = 42.227 km/hr
The Velocity = 7.854 km/hr
The Velocity = 9.605 km/hr
The Velocity = 8.559 km/hr
The Velocity = 12.760 km/hr
The Velocity = 24.386 km/hr
The Velocity = 12.442 km/hr
[INFO] elasped time: 366.05
[INFO] approx. FPS: 4.97
